In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

glueContext = GlueContext(SparkContext.getOrCreate())

# neste exemplo o arquivo anterior particionou por agencia, depois incluimos os outros dois ids, a agencia vou usar tb para deixar bem leve

# serao dois processos paralelos por digitos, em duas faixas de agencia

In [ ]:
import datetime
import json
from pprint import pprint
from pyspark.sql.functions import lit
import boto3
import pyspark.sql.functions as f 
import sys
from awsglue.job import Job
from awsglue.transforms import *
from awsglue.context import GlueContext
from pyspark.context import SparkContext
from awsglue.utils import getResolvedOptions
from awsglue.dynamicframe import DynamicFrame
from pyspark.sql.types import ArrayType, StructField, StructType, StringType, IntegerType
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType,DateType
from pyspark.sql.functions import lit
from pyspark.sql.functions import col
from pyspark.sql.types import StringType,BooleanType
from awsglue.dynamicframe import DynamicFrame
enviroment = 'der'

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 0 a partir agencia 5
#job_seletor_digito_0_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '0'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()


#precisamos alinhar o salvamento no PLANO para o agrupador

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 1 a partir agencia 5
#job_seletor_digito_1_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '1'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()


#precisamos alinhar o salvamento no PLANO para o agrupador

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 2 a partir agencia 5
#job_seletor_digito_2_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '2'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()



#precisamos alinhar o salvamento no PLANO para o agrupador

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 3 a partir agencia 5
#job_seletor_digito_3_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '3'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()



#precisamos alinhar o salvamento no PLANO para o agrupador

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 4 a partir agencia 5
#job_seletor_digito_4_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '4'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()



#precisamos alinhar o salvamento no PLANO para o agrupador

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 5 a partir agencia 5
#job_seletor_digito_5_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '5'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()



#precisamos alinhar o salvamento no PLANO para o agrupador

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 6 a partir agencia 5
#job_seletor_digito_6_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '6'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()



#precisamos alinhar o salvamento no PLANO para o agrupador

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 7 a partir agencia 5
#job_seletor_digito_7_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '7'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()



#precisamos alinhar o salvamento no PLANO para o agrupador

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 8 a partir agencia 5
#job_seletor_digito_8_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '8'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()



#precisamos alinhar o salvamento no PLANO para o agrupador

In [ ]:
# Digitos de 0 a 9  a partir agencia 5
### Digitos de 9 a partir agencia 5
#job_seletor_digito_19_range_agencia_a_partir_5.py
database = "banco_bonus"
tbcadastral = "tb_topicos_cadastral"
tbbonus = "tb_boletos_filtrador"
digito = '9'
faixa_agencia_ate_4 = '4'
faixa_agencia_a_partir_de_5 = '5'
partition_cadastral_digito ="numero_digito_verificador_= '{}' and partition_agencia >='{}'".format(digito,faixa_agencia_a_partir_de_5)

print(partition_cadastral_digito)
partition_day_cobranca = "%s%s%s%s%s%s%s"  % ("numero_digito_verificador= '{}' and partition_agencia <='{}'and year='".format(digito,faixa_agencia_ate_4),year_yesterday_referencia,"' and month='",month_yesterday_referencia,"'and day='",day_yesterday_referencia,"'")
print(partition_day_cobranca)

datasource_cadastral_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbcadastral,
    transformation_ctx="datasource_cadastral_digito",
    push_down_predicate=partition_cadastral_digito,
    additional_options={"catalogPartitionPredicate":partition_cadastral_digito} )
datasource_bonus_digito = glueContext.create_dynamic_frame.from_catalog(
    database=database, 
    table_name=tbbonus,
    transformation_ctx="datasource_bonus_digito",
    push_down_predicate=partition_day_cobranca,
    additional_options={"catalogPartitionPredicate":partition_day_cobranca} )

datasource_bonus_digito = datasource_bonus_digito.toDF()
datasource_bonus_digito.show(10)
datasource_cadastral_digito = datasource_cadastral_digito.toDF()
datasource_cadastral_digito.show(10)


# join 
datasource_bonus_digito.join(datasource_cadastral_digito, \
(datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
(datasource_bonus_digito.numero_conta_corrente == datasource_cadastral_digito.numero_conta_corrente_)&
(datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
).select('cnpj','numero_digito_verificador','numero_agencia','type','numero_conta_corrente','valor_pago_cobranca','codigo_segmento_').show()

# #debugar os que nao achar no athena
# # SELECT cnpj, numero_agencia_, numero_conta_corrente_, numero_digito_verificador, numero_digito_verificador_, numero_agencia, numero_conta_corrente FROM "banco_bonus"."tb___topicos_cadastral" inner join "tb_boletos_conversor"
# # on 
# # (tb___topicos_cadastral.numero_agencia_ = tb_boletos_conversor.numero_agencia 
# # and tb___topicos_cadastral.partition_agencia = tb_boletos_conversor.partition_agencia
# # and tb___topicos_cadastral.numero_digito_verificador_ = tb_boletos_conversor.numero_digito_verificador
# # )

# # debugar aqui os que nao achar
# # datasource_bonus_digito.join(datasource_cadastral_digito, \
# # (datasource_bonus_digito.numero_agencia == datasource_cadastral_digito.numero_agencia_)& 
# # (datasource_bonus_digito.partition_agencia == datasource_cadastral_digito.partition_agencia)&
# # (datasource_bonus_digito.numero_digito_verificador == datasource_cadastral_digito.numero_digito_verificador_)
# # ).show()



#precisamos alinhar o salvamento no PLANO para o agrupador